In [13]:
import torch
import torch.nn as nn #import neural network class
from torch.utils.data import DataLoader #helps in creating batchs
from torchvision import datasets #download data
from torchvision.transforms import ToTensor
import torch.nn.functional as F

In [2]:
## download data
training_data = datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
test_data = datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())

100%|██████████| 26421880/26421880 [00:02<00:00, 12332421.56it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 208962.26it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3906213.69it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5556427.43it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size) #inputdata and Outputdata
test_dataloader = DataLoader(test_data,batch_size=batch_size)

In [6]:
for X,y in train_dataloader:
  print (X.shape)
  print (y.shape)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [7]:
len(train_dataloader)

938

In [8]:
len(test_dataloader)

157

In [9]:
## device setup

device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
device

'cuda'

In [17]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork,self).__init__() ##it copies the initializations from the nn.Module class
    self.flatten = nn.Flatten() #28x28x1 is converted to 764x1
    self.linear1 = nn.Linear(28*28,512)
    self.linear2 = nn.Linear(512,512)
    self.output = nn.Linear(512,10) #10 class classification problem

  def forward(self,x):
    x = self.flatten(x)
    x = self.linear1(x)
    x = F.relu(x)
    x = self.linear2(x)
    x = F.relu(x)
    out = self.output(x) #We didn't use softmax here
    return out

In [18]:
model = NeuralNetwork().to(device)
print (model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (output): Linear(in_features=512, out_features=10, bias=True)
)


In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.0001)

In [23]:
## define the training loop

def train(dataloader,model,loss_fn,optimizer):
  model.train()
  for batch,(X,y) in enumerate(dataloader):
    X = X.to(device)
    y = y.to(device)

    pred = model(X)
    loss = loss_fn(pred,y)

    ##backpropogation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      print (f'Loss : {loss.item()}')

In [24]:
epochs = 5

for t in range(epochs):
  print ('Epoch---------------------------')
  train(train_dataloader,model,loss_fn,optimizer)

Epoch---------------------------
Loss : 2.2761874198913574
Loss : 2.276379108428955
Loss : 2.26177716255188
Loss : 2.2697107791900635
Loss : 2.2635293006896973
Loss : 2.2555432319641113
Loss : 2.26572847366333
Loss : 2.2492239475250244
Loss : 2.2588818073272705
Loss : 2.252143383026123
Epoch---------------------------
Loss : 2.2624928951263428
Loss : 2.263683557510376
Loss : 2.2459254264831543
Loss : 2.2572076320648193
Loss : 2.2503719329833984
Loss : 2.2390666007995605
Loss : 2.2528159618377686
Loss : 2.233665704727173
Loss : 2.2453207969665527
Loss : 2.2378625869750977
Epoch---------------------------
Loss : 2.248300552368164
Loss : 2.250559091567993
Loss : 2.2296900749206543
Loss : 2.244152069091797
Loss : 2.2365660667419434
Loss : 2.221919298171997
Loss : 2.239365339279175
Loss : 2.217560052871704
Loss : 2.2310447692871094
Loss : 2.22286319732666
Epoch---------------------------
Loss : 2.2335691452026367
Loss : 2.236809253692627
Loss : 2.212857246398926
Loss : 2.230506658554077
Los